In [1]:
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Settings
dataset_name='citeseer'
args.dataset=dataset_name
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)
nodesize = features.shape[0]

# Some preprocessing
features_tmp=features.copy()
features = preprocess_features(features).A
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
# Setup Surrogate model
idx_train=np.array(np.where(train_mask==1)).tolist()[0]
idx_val=np.array(np.where(val_mask==1)).tolist()[0]
idx_unlabeled=np.array(np.where(test_mask==1)).tolist()[0]
surrogate = GCN(nfeat=features.shape[1], nclass=single_label.max().item()+1,
                nhid=256, dropout=0, with_relu=False, with_bias=False, device='cpu').to('cpu')
surrogate.fit(features, adj, single_label, idx_train, idx_val, patience=100)
# Setup Attack Model
model = Metattack(surrogate, nnodes=adj.shape[0], feature_shape=features.shape,
        attack_structure=True, attack_features=False, device='cpu', lambda_=0).to('cpu')
# Attack
model.attack(features, adj, single_label, idx_train, idx_unlabeled, n_perturbations=20, ll_constraint=False)
modified_adj = model.modified_adj
# print(adj)
# print("shiy")
# print(modified_adj)

/home/zsp/ljx/PTDNet/utils.py:221: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()
Perturbing graph:   0%|          | 0/20 [00:00<?, ?it/s]

GCN loss on unlabled data: 1.6932793855667114
GCN acc on unlabled data: 0.67
attack loss: 1.6872400045394897


Perturbing graph:   5%|▌         | 1/20 [07:01<2:13:20, 421.09s/it]

GCN loss on unlabled data: 1.69221830368042
GCN acc on unlabled data: 0.678
attack loss: 1.6853837966918945


Perturbing graph:  10%|█         | 2/20 [13:59<2:05:50, 419.49s/it]

...

Perturbing graph:  90%|█████████ | 18/20 [1:47:51<09:06, 273.25s/it]

GCN loss on unlabled data: 1.712292194366455
GCN acc on unlabled data: 0.661
attack loss: 1.706063151359558


Perturbing graph:  95%|█████████▌| 19/20 [1:52:19<04:31, 271.54s/it]

GCN loss on unlabled data: 1.714308261871338
GCN acc on unlabled data: 0.67
attack loss: 1.707374930381775


Perturbing graph: 100%|██████████| 20/20 [1:56:41<00:00, 350.10s/it]


In [2]:
modified_adj=sp.csr_array(modified_adj.int())

In [3]:
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

# Settings
args.dataset=dataset_name
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)

nodesize = features.shape[0]

# Some preprocessing
features = preprocess_features(features)
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)



best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000
import time
begin = time.time()

model = PTDNetGCN(input_dim=features.shape[1], output_dim=y_train.shape[1])
model.set_fea_adj(np.array(range(adj.shape[0])), features_tensor, adj_tensor)

best_epoch = 0
curr_step = 0
best_val_acc = 0

for epoch in range(args.epochs):
    temperature = max(0.05,args.init_temperature * pow(args.temperature_decay, epoch))
    with tf.GradientTape() as tape:
        preds = []
        for l in range(args.outL):
            output = model.call(temperature,training=True)
            preds.append(tf.expand_dims(output,0))
        all_preds = tf.concat(preds,axis=0)
        mean_preds = tf.reduce_mean(preds,axis=0)
        consistency_loss = tf.nn.l2_loss(mean_preds-all_preds)

        cross_loss = masked_softmax_cross_entropy(mean_preds, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        lossl0 = model.lossl0(temperature)
        #nuclear = model.my_nuclear()
        nuclear = model.nuclear()
        loss = cross_loss + args.weight_decay*lossL2 + args.lambda1*lossl0 + args.lambda3*nuclear + args.coff_consis*consistency_loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call(None, training=False)
    edges_volumn = tf.reduce_sum(model.maskes[0])
    print('edge_vol',edges_volumn.numpy())

    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)
    if val_acc > best_val_acc:
        curr_step = 0
        best_epoch = epoch
        best_val_acc = val_acc
        best_val_loss= val_loss
        if val_acc>best_val_acc_trail:
            best_test_acc = test_acc
            best_val_acc_trail = val_acc
    else:
        curr_step +=1
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(train_acc), "val_acc=", "{:.5f}".format(val_acc),"best_val_acc_trail=", "{:.5f}".format(best_val_acc_trail),
      "test_acc=", "{:.5f}".format(best_test_acc))

    if curr_step > args.early_stop:
        print("Early stopping...")
        break
    end = time.time()
    print('time ',(end-begin))


/home/zsp/ljx/PTDNet/utils.py:221: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


edge_vol 4661.3887
Epoch: 0001 train_loss= 1.79182 val_loss= 1.79108 train_acc= 0.40000 val_acc= 0.27000 best_val_acc_trail= 0.27000 test_acc= 0.26900
time  6.624025106430054
edge_vol 4649.8633
Epoch: 0002 train_loss= 1.78954 val_loss= 1.79035 train_acc= 0.67500 val_acc= 0.33600 best_val_acc_trail= 0.33600 test_acc= 0.35000
time  7.992251873016357
edge_vol 4637.8774
Epoch: 0003 train_loss= 1.78716 val_loss= 1.78960 train_acc= 0.82500 val_acc= 0.39800 best_val_acc_trail= 0.39800 test_acc= 0.41000
time  9.131654262542725
edge_vol 4625.4824
Epoch: 0004 train_loss= 1.78475 val_loss= 1.78880 train_acc= 0.88333 val_acc= 0.44000 best_val_acc_trail= 0.44000 test_acc= 0.47800
time  10.288454532623291
edge_vol 4612.746
Epoch: 0005 train_loss= 1.78233 val_loss= 1.78794 train_acc= 0.92500 val_acc= 0.47600 best_val_acc_trail= 0.47600 test_acc= 0.51900
time  11.072489738464355
edge_vol 4599.7676
Epoch: 0006 train_loss= 1.77977 val_loss= 1.78700 train_acc= 0.91667 val_acc= 0.52800 best_val_acc_trail=

In [4]:
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

# Settings
args.dataset=dataset_name

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)

nodesize = features.shape[0]

# Some preprocessing
features = preprocess_features(features)
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)



best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000
import time
begin = time.time()

model = PTDNetGCN(input_dim=features.shape[1], output_dim=y_train.shape[1])
model.set_fea_adj(np.array(range(adj.shape[0])), features_tensor, adj_tensor)

best_epoch = 0
curr_step = 0
best_val_acc = 0

for epoch in range(args.epochs):
    temperature = max(0.05,args.init_temperature * pow(args.temperature_decay, epoch))
    with tf.GradientTape() as tape:
        preds = []
        for l in range(args.outL):
            output = model.call(temperature,training=True)
            preds.append(tf.expand_dims(output,0))
        all_preds = tf.concat(preds,axis=0)
        mean_preds = tf.reduce_mean(preds,axis=0)
        consistency_loss = tf.nn.l2_loss(mean_preds-all_preds)

        cross_loss = masked_softmax_cross_entropy(mean_preds, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        lossl0 = model.lossl0(temperature)
        nuclear = model.my_nuclear()
        #nuclear = model.nuclear()
        loss = cross_loss + args.weight_decay*lossL2 + args.lambda1*lossl0 + args.lambda3*nuclear + args.coff_consis*consistency_loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call(None, training=False)
    edges_volumn = tf.reduce_sum(model.maskes[0])
    print('edge_vol',edges_volumn.numpy())

    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)
    if val_acc > best_val_acc:
        curr_step = 0
        best_epoch = epoch
        best_val_acc = val_acc
        best_val_loss= val_loss
        if val_acc>best_val_acc_trail:
            best_test_acc = test_acc
            best_val_acc_trail = val_acc
    else:
        curr_step +=1
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(train_acc), "val_acc=", "{:.5f}".format(val_acc),"best_val_acc_trail=", "{:.5f}".format(best_val_acc_trail),
      "test_acc=", "{:.5f}".format(best_test_acc))

    if curr_step > args.early_stop:
        print("Early stopping...")
        break
    end = time.time()
    print('time ',(end-begin))


edge_vol 4682.702
Epoch: 0001 train_loss= 1.79186 val_loss= 1.79155 train_acc= 0.40833 val_acc= 0.21000 best_val_acc_trail= 0.21000 test_acc= 0.19300
time  3.4570846557617188
edge_vol 4688.2783
Epoch: 0002 train_loss= 1.78976 val_loss= 1.79092 train_acc= 0.64167 val_acc= 0.29800 best_val_acc_trail= 0.29800 test_acc= 0.26500
time  4.128012418746948
edge_vol 4689.879
Epoch: 0003 train_loss= 1.78762 val_loss= 1.79027 train_acc= 0.79167 val_acc= 0.35400 best_val_acc_trail= 0.35400 test_acc= 0.32400
time  4.78596568107605
edge_vol 4687.903
Epoch: 0004 train_loss= 1.78541 val_loss= 1.78959 train_acc= 0.83333 val_acc= 0.38600 best_val_acc_trail= 0.38600 test_acc= 0.37500
time  5.555476903915405
edge_vol 4683.5527
Epoch: 0005 train_loss= 1.78323 val_loss= 1.78885 train_acc= 0.88333 val_acc= 0.43600 best_val_acc_trail= 0.43600 test_acc= 0.42400
time  6.323407411575317
edge_vol 4677.861
Epoch: 0006 train_loss= 1.78096 val_loss= 1.78805 train_acc= 0.90000 val_acc= 0.47000 best_val_acc_trail= 0.47

/home/zsp/ljx/PTDNet/models.py:14: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


edge_vol 9268.0
Epoch: 0325 train_loss= 0.46772 val_loss= 1.14369 train_acc= 0.98333 val_acc= 0.71200 best_val_acc_trail= 0.71600 test_acc= 0.71000
time  250.66522645950317
edge_vol 9268.0
Epoch: 0326 train_loss= 0.46618 val_loss= 1.14243 train_acc= 0.98333 val_acc= 0.71200 best_val_acc_trail= 0.71600 test_acc= 0.71000
time  251.54175186157227
edge_vol 9268.0
Epoch: 0327 train_loss= 0.46366 val_loss= 1.14110 train_acc= 0.98333 val_acc= 0.71400 best_val_acc_trail= 0.71600 test_acc= 0.71000
time  252.39025378227234
edge_vol 9268.0
Epoch: 0328 train_loss= 0.46169 val_loss= 1.13987 train_acc= 0.98333 val_acc= 0.71400 best_val_acc_trail= 0.71600 test_acc= 0.71000
time  253.1144242286682
edge_vol 9268.0
Epoch: 0329 train_loss= 0.46017 val_loss= 1.13860 train_acc= 0.98333 val_acc= 0.71400 best_val_acc_trail= 0.71600 test_acc= 0.71000
time  253.8023030757904
edge_vol 9268.0
Epoch: 0330 train_loss= 0.45894 val_loss= 1.13738 train_acc= 0.98333 val_acc= 0.71400 best_val_acc_trail= 0.71600 test_ac

In [5]:
from config import args
import tensorflow as tf
import time
from utils import *
from models import GCN_dropedge
from metrics import *

# Settings
optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
args.dataset=dataset_name

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj

tuple_adj = sparse_to_tuple(adj.tocoo())
adj_tensor = tf.SparseTensor(*tuple_adj)

features = preprocess_features(features)

model = GCN_dropedge(input_dim=features.shape[1], output_dim=y_train.shape[1], adj=adj_tensor)


features_tensor = tf.convert_to_tensor(features,dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=tf.float32)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=tf.float32)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=tf.float32)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc = 0
best_val_loss = 10000


curr_step = 0
for epoch in range(args.epochs):

    with tf.GradientTape() as tape:
        output = model.call((features_tensor),training=True)
        cross_loss = masked_softmax_cross_entropy(output, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        loss = cross_loss + args.weight_decay*lossL2
        grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call((features_tensor), training=False)
    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)

    if val_acc > best_val_acc:
        curr_step = 0
        best_test_acc = test_acc
        best_val_acc = val_acc
        best_val_loss= val_loss
        # Print results

    else:
        curr_step +=1
    if curr_step > args.early_stop:
        print("Early stopping...")
        break

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(val_acc), "val_acc=", "{:.5f}".format(val_acc),
      "test_acc=", "{:.5f}".format(best_test_acc))

Epoch: 0001 train_loss= 1.79135 val_loss= 1.79116 train_acc= 0.19800 val_acc= 0.19800 test_acc= 0.22600
Epoch: 0002 train_loss= 1.78920 val_loss= 1.79046 train_acc= 0.33000 val_acc= 0.33000 test_acc= 0.35700
Epoch: 0003 train_loss= 1.78704 val_loss= 1.78973 train_acc= 0.44600 val_acc= 0.44600 test_acc= 0.46900
Epoch: 0004 train_loss= 1.78487 val_loss= 1.78897 train_acc= 0.51200 val_acc= 0.51200 test_acc= 0.53500
Epoch: 0005 train_loss= 1.78265 val_loss= 1.78815 train_acc= 0.56000 val_acc= 0.56000 test_acc= 0.56900
Epoch: 0006 train_loss= 1.78035 val_loss= 1.78726 train_acc= 0.60200 val_acc= 0.60200 test_acc= 0.59800
Epoch: 0007 train_loss= 1.77793 val_loss= 1.78630 train_acc= 0.62400 val_acc= 0.62400 test_acc= 0.61500
Epoch: 0008 train_loss= 1.77536 val_loss= 1.78526 train_acc= 0.64400 val_acc= 0.64400 test_acc= 0.63000
Epoch: 0009 train_loss= 1.77264 val_loss= 1.78413 train_acc= 0.65600 val_acc= 0.65600 test_acc= 0.64100
Epoch: 0010 train_loss= 1.76974 val_loss= 1.78292 train_acc= 0.6